# PriorityQueue - Async Priority Queue

PriorityQueue is an async priority queue built on heapq + anyio.Condition:

**Core Features:**
- **Priority Ordering**: Items retrieved by lowest value first (min-heap)
- **Async Operations**: All operations are async (including nowait variants)
- **Blocking/Non-Blocking**: Support for both put/get (blocking) and put_nowait/get_nowait
- **Maxsize Support**: Optional capacity limit with automatic blocking
- **Thread-Safe**: Built on anyio.Condition for cross-task synchronization
- **Exception Handling**: QueueEmpty and QueueFull exceptions for error cases

In [1]:
import anyio

from lionherd_core.libs.concurrency import PriorityQueue, QueueEmpty, QueueFull, sleep

## 1. Basic Construction

Create queue with optional maxsize. Default (0) means unlimited.

In [2]:
# Unlimited queue
unlimited = PriorityQueue()
print(f"Maxsize: {unlimited.maxsize} (unlimited)")
print(f"Initial size: {unlimited.qsize()}")
print(f"Empty: {unlimited.empty()}")

# Limited capacity
limited = PriorityQueue(maxsize=3)
print(f"\nLimited maxsize: {limited.maxsize}")
print(f"Full: {limited.full()}")

Maxsize: 0 (unlimited)
Initial size: 0
Empty: True

Limited maxsize: 3
Full: False


## 2. Putting Items with Priority

Items are tuples with priority as first element. Lower values = higher priority.

In [3]:
queue = PriorityQueue()

# Put items with different priorities (priority, data)
await queue.put((3, "Low priority task"))
await queue.put((1, "High priority task"))
await queue.put((2, "Medium priority task"))

print(f"Queue size after puts: {queue.qsize()}")
print(f"Empty: {queue.empty()}")

Queue size after puts: 3
Empty: False


## 3. Getting Items (Priority Order)

Items are retrieved in priority order (lowest value first).

In [4]:
# Get items - should come out in priority order
first = await queue.get()
print(f"First: {first}")  # (1, "High priority task")

second = await queue.get()
print(f"Second: {second}")  # (2, "Medium priority task")

third = await queue.get()
print(f"Third: {third}")  # (3, "Low priority task")

print(f"\nQueue size after gets: {queue.qsize()}")
print(f"Empty: {queue.empty()}")

First: (1, 'High priority task')
Second: (2, 'Medium priority task')
Third: (3, 'Low priority task')

Queue size after gets: 0
Empty: True


## 4. Non-Blocking Operations

nowait methods raise exceptions instead of blocking. Note: Unlike asyncio, these are async.

In [5]:
queue = PriorityQueue()

# put_nowait - succeeds on non-full queue
await queue.put_nowait((1, "Item 1"))
await queue.put_nowait((2, "Item 2"))
print(f"After put_nowait: size={queue.qsize()}")

# get_nowait - succeeds on non-empty queue
item = await queue.get_nowait()
print(f"Got via nowait: {item}")
print(f"After get_nowait: size={queue.qsize()}")

After put_nowait: size=2
Got via nowait: (1, 'Item 1')
After get_nowait: size=1


## 5. Queue State Checks

qsize(), empty(), and full() are synchronous but racy. Use for monitoring only.

In [6]:
queue = PriorityQueue(maxsize=2)

# Check initial state
print(f"Initial - size: {queue.qsize()}, empty: {queue.empty()}, full: {queue.full()}")

# Add one item
await queue.put((1, "Item 1"))
print(f"After 1 item - size: {queue.qsize()}, empty: {queue.empty()}, full: {queue.full()}")

# Fill to capacity
await queue.put((2, "Item 2"))
print(f"After 2 items - size: {queue.qsize()}, empty: {queue.empty()}, full: {queue.full()}")

# Empty queue
await queue.get()
await queue.get()
print(f"After clearing - size: {queue.qsize()}, empty: {queue.empty()}, full: {queue.full()}")

Initial - size: 0, empty: True, full: False
After 1 item - size: 1, empty: False, full: False
After 2 items - size: 2, empty: False, full: True
After clearing - size: 0, empty: True, full: False


## 6. Maxsize Blocking Behavior

put() blocks when queue is full. get() blocks when queue is empty.

In [7]:
queue = PriorityQueue(maxsize=2)
results = []


async def producer():
    """Try to put 3 items into queue with maxsize=2 (will block on 3rd)"""
    results.append("Producer: putting item 1")
    await queue.put((1, "Item 1"))
    results.append("Producer: put item 1")

    results.append("Producer: putting item 2")
    await queue.put((2, "Item 2"))
    results.append("Producer: put item 2")

    results.append("Producer: putting item 3 (will block until space available)")
    await queue.put((3, "Item 3"))
    results.append("Producer: put item 3 (unblocked!)")


async def consumer():
    """Wait a bit, then consume one item to unblock producer"""
    await sleep(0.01)
    results.append("Consumer: getting item (will unblock producer)")
    item = await queue.get()
    results.append(f"Consumer: got {item}")


# Run concurrently using anyio task group - producer blocks until consumer makes space
async with anyio.create_task_group() as tg:
    tg.start_soon(producer)  # Pass callable, not coroutine
    tg.start_soon(consumer)

print("Execution sequence:")
for r in results:
    print(f"  {r}")

print(f"\nFinal queue size: {queue.qsize()}")

Execution sequence:
  Producer: putting item 1
  Producer: put item 1
  Producer: putting item 2
  Producer: put item 2
  Producer: putting item 3 (will block until space available)
  Consumer: getting item (will unblock producer)
  Consumer: got (1, 'Item 1')
  Producer: put item 3 (unblocked!)

Final queue size: 2


## 7. Exception Handling

QueueEmpty raised by get_nowait() on empty queue. QueueFull raised by put_nowait() on full queue.

In [8]:
empty_queue = PriorityQueue()

# QueueEmpty exception
try:
    await empty_queue.get_nowait()
except QueueEmpty as e:
    print(f"✓ QueueEmpty caught: {e}")

# QueueFull exception
full_queue = PriorityQueue(maxsize=1)
await full_queue.put((1, "Item 1"))  # Fill queue

try:
    await full_queue.put_nowait((2, "Item 2"))
except QueueFull as e:
    print(f"✓ QueueFull caught: {e}")

✓ QueueEmpty caught: Queue is empty
✓ QueueFull caught: Queue is full


## 8. Priority Ordering with Complex Items

When using complex items with duplicate priorities, add a counter for tie-breaking. Pattern: `(priority, counter, data)`

In [9]:
import itertools

queue = PriorityQueue()
counter = itertools.count()  # Unique counter for tie-breaking

# Define tasks with priority levels
# Priority: 0 = critical, 1 = high, 2 = medium, 3 = low
# Pattern: (priority, counter, data) - counter breaks ties
tasks = [
    (2, next(counter), {"name": "Update docs", "duration": "30m"}),
    (0, next(counter), {"name": "Fix production bug", "duration": "2h"}),
    (3, next(counter), {"name": "Refactor old code", "duration": "4h"}),
    (1, next(counter), {"name": "Review PR", "duration": "1h"}),
    (0, next(counter), {"name": "Security patch", "duration": "1h"}),
]

# Put tasks in random order
for task in tasks:
    await queue.put(task)

print("Tasks retrieved in priority order:")
print()

while not queue.empty():
    priority, _, task_data = await queue.get()  # Unpack counter but ignore it
    priority_label = ["CRITICAL", "HIGH", "MEDIUM", "LOW"][priority]
    print(f"  [{priority_label}] {task_data['name']} ({task_data['duration']})")

Tasks retrieved in priority order:

  [CRITICAL] Fix production bug (2h)
  [CRITICAL] Security patch (1h)
  [HIGH] Review PR (1h)
  [MEDIUM] Update docs (30m)
  [LOW] Refactor old code (4h)


## 9. Concurrent Producer-Consumer Pattern

Multiple producers and consumers working concurrently.

In [10]:
queue = PriorityQueue(maxsize=5)
consumed = []


async def producer(producer_id: int, num_items: int):
    """Produce items with varying priorities"""
    for i in range(num_items):
        priority = i % 3  # Cycle through priorities 0, 1, 2
        await queue.put((priority, f"P{producer_id}-Item{i}"))
        await sleep(0.001)  # Simulate work


async def consumer(consumer_id: int, num_items: int):
    """Consume items and track what was processed"""
    for _ in range(num_items):
        priority, item = await queue.get()
        consumed.append((consumer_id, priority, item))
        await sleep(0.001)  # Simulate processing


# Run 2 producers and 2 consumers concurrently using anyio task group
async with anyio.create_task_group() as tg:
    tg.start_soon(producer, 1, 4)  # Args go as separate parameters
    tg.start_soon(producer, 2, 4)
    tg.start_soon(consumer, 1, 4)
    tg.start_soon(consumer, 2, 4)

print("Consumed items (consumer_id, priority, item):")
for consumer_id, priority, item in consumed:
    print(f"  Consumer {consumer_id}: [{priority}] {item}")

print(f"\nFinal queue size: {queue.qsize()}")

Consumed items (consumer_id, priority, item):
  Consumer 1: [0] P1-Item0
  Consumer 2: [0] P2-Item0
  Consumer 1: [1] P1-Item1
  Consumer 2: [1] P2-Item1
  Consumer 1: [2] P1-Item2
  Consumer 2: [2] P2-Item2
  Consumer 1: [0] P1-Item3
  Consumer 2: [0] P2-Item3

Final queue size: 0


## 10. Wait vs Nowait Comparison

Demonstrate the difference between blocking and non-blocking operations.

In [11]:
queue = PriorityQueue()
logs = []


async def blocking_get():
    """Blocks until item available"""
    logs.append("blocking_get: waiting for item...")
    item = await queue.get()
    logs.append(f"blocking_get: received {item}")


async def delayed_put():
    """Put item after delay"""
    await sleep(0.01)
    logs.append("delayed_put: putting item")
    await queue.put((1, "delayed item"))
    logs.append("delayed_put: done")


# Run concurrently using anyio task group - blocking_get waits for delayed_put
async with anyio.create_task_group() as tg:
    tg.start_soon(blocking_get)  # Pass callable, not coroutine
    tg.start_soon(delayed_put)

print("Blocking operation sequence:")
for log in logs:
    print(f"  {log}")

# Now demonstrate nowait failure
queue2 = PriorityQueue()
print("\nNowait operation:")
try:
    item = await queue2.get_nowait()
    print(f"  Got item: {item}")
except QueueEmpty:
    print("  ✓ get_nowait raised QueueEmpty immediately (no blocking)")

Blocking operation sequence:
  blocking_get: waiting for item...
  delayed_put: putting item
  delayed_put: done
  blocking_get: received (1, 'delayed item')

Nowait operation:
  ✓ get_nowait raised QueueEmpty immediately (no blocking)


## Summary Checklist

**PriorityQueue Essentials:**
- ✅ Min-heap priority ordering (lowest value = highest priority)
- ✅ Async operations (all methods are async)
- ✅ Blocking put/get (wait for space/items)
- ✅ Non-blocking put_nowait/get_nowait (raise exceptions)
- ✅ Optional maxsize with automatic blocking
- ✅ Thread-safe via anyio.Condition
- ✅ State checks (qsize/empty/full) for monitoring
- ✅ Exception handling (QueueEmpty/QueueFull)
- ✅ Concurrent producer-consumer support

**Key Differences from asyncio.PriorityQueue:**
- nowait methods are async (not sync)
- Built on anyio.Condition (not asyncio.Condition)
- Cross-backend compatibility (asyncio/trio)

**Next Steps:**
- See `_primitives.py` for Condition, Lock, Event
- See `_executor.py` for ThreadPoolExecutor patterns
- See `_queue_mixin.py` for queue-based collection utilities